In [1]:
# Markdown is not rendered in PyCharm.

# DeepBird

## Xeno Canto

This is a [website](https://www.xeno-canto.org) containing bird song recordings and classification. They have shared part of their [collection](https://www.gbif.org/dataset/b1047888-ae52-4179-9dd5-5448ea342a24) to [Global Biodiversity Information Facility](https://gbif.org)

### Terms of use

It is ok to download some files but we still need to contact for downloading the complete set. See the [terms](https://www.xeno-canto.org/about/terms)

## About this project

We want to learn the individual bird sound and try to learn their songs.

For this to happen we need to

- download the zip containing a csv from GBIF.
- download samples with their annotations.
- process the samples into a spectrogram and extract regions of interest.
- learn the extracted regions.

## Fetching

It seems the first item is `https://www.xeno-canto.org/1` and currently `https://www.xeno-canto.org/460846` the last.

As we have not contacted them yet we collect 5 samples each time we run this page.

In [2]:
import project;

# In case you want to have proces data somewhere else change it
data_dir = project.get_project_dir() + 'data/'
data_dir = '/media/clemens/Maxtor/xeno-canto/'
project.set_data_dir(data_dir)

num_files = 5

project.print_stats()

Project   : /home/clemens/Documents/projects/PyDeepSqueak/
Data      : /media/clemens/Maxtor/xeno-canto/
GBIF      : /media/clemens/Maxtor/xeno-canto/gbif/
sample    : /media/clemens/Maxtor/xeno-canto/xc/
fragments : /media/clemens/Maxtor/xeno-canto/fragments/


In [3]:
import gbif

gbif.set_gbif_dir(project.get_gbif_dir())

gbif_id = '0025627-181108115102211'

In [4]:
gbif_csv = gbif.get_data()

Already downloaded zip 0025627-181108115102211


In [5]:
import pandas as pd

df = pd.read_csv(gbif_csv, sep='\t')

project.fix_gbif_df(df)

# Do we have data?
df.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue,XC_ID
0,1934871083,b1047888-ae52-4179-9dd5-5448ea342a24,http://data.biodiversitydata.nl/xeno-canto/obs...,Animalia,Chordata,Aves,Passeriformes,Parulidae,Seiurus,Seiurus aurocapilla,...,NaN,CC_BY_NC_4_0,Mike Nelson,Mike Nelson,NaN,NaN,2019-01-03T10:18:26.934Z,SOUND;STILLIMAGE,NaN,100119
1,1934871084,b1047888-ae52-4179-9dd5-5448ea342a24,http://data.biodiversitydata.nl/xeno-canto/obs...,Animalia,Chordata,Aves,Passeriformes,Turdidae,Catharus,Catharus guttatus,...,NaN,CC_BY_NC_4_0,Mike Nelson,Mike Nelson,NaN,NaN,2019-01-03T10:18:26.933Z,SOUND;STILLIMAGE,NaN,100113
2,1934871085,b1047888-ae52-4179-9dd5-5448ea342a24,http://data.biodiversitydata.nl/xeno-canto/obs...,Animalia,Chordata,Aves,Passeriformes,Parulidae,Setophaga,Setophaga ruticilla,...,NaN,CC_BY_NC_4_0,Mike Nelson,Mike Nelson,NaN,NaN,2019-01-03T10:18:26.936Z,SOUND;STILLIMAGE,NaN,100082
3,1934871086,b1047888-ae52-4179-9dd5-5448ea342a24,http://data.biodiversitydata.nl/xeno-canto/obs...,Animalia,Chordata,Aves,Passeriformes,Vireonidae,Vireo,Vireo gilvus,...,NaN,CC_BY_NC_4_0,Andrew Spencer,Andrew Spencer,NaN,NaN,2019-01-03T10:18:27.014Z,SOUND;STILLIMAGE,NaN,100053
4,1934871087,b1047888-ae52-4179-9dd5-5448ea342a24,http://data.biodiversitydata.nl/xeno-canto/obs...,Animalia,Chordata,Aves,Passeriformes,Icteridae,Dolichonyx,Dolichonyx oryzivorus,...,NaN,CC_BY_NC_4_0,Mike Nelson,Mike Nelson,NaN,NaN,2019-01-03T10:18:26.932Z,SOUND;STILLIMAGE,NaN,100089


In [6]:
# We have some classification of the birg
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170041 entries, 0 to 170040
Data columns (total 46 columns):
gbifID                           170041 non-null int64
datasetKey                       170041 non-null object
occurrenceID                     170041 non-null object
kingdom                          170041 non-null object
phylum                           168749 non-null object
class                            168749 non-null object
order                            168749 non-null object
family                           168749 non-null object
genus                            168749 non-null object
species                          168535 non-null object
infraspecificEpithet             39389 non-null object
taxonRank                        170041 non-null object
scientificName                   170041 non-null object
countryCode                      169447 non-null object
locality                         170040 non-null object
publishingOrgKey                 170041 non-null ob

In [7]:
# occurenceID references to http://data.biodiversitydata.nl/xeno-canto/observation/XC######
# fetching this resource redirects to the page and not the download link

df['occurrenceID'].head().map(lambda x: x.rsplit('/', 1)[1])

0    XC100119
1    XC100113
2    XC100082
3    XC100053
4    XC100089
Name: occurrenceID, dtype: object

In [8]:
df['XC_ID'].head()

0    100119
1    100113
2    100082
3    100053
4    100089
Name: XC_ID, dtype: object

In [14]:
xc_id='100082'

print('FP:', project.get_fragments_path(df, xc_id))
import split


args = dict(split.defaults)

args['silence_threshold'] = 0.01
args['min_silence_length'] = 1.0
args['dry_run'] = False

print('Data:', project.get_data_dir())
print('Fragments:',project.get_fragments_dir())
project.build_fragments(df.head(num_files), args)

  9%|▉         | 136/1510 [00:00<00:01, 1357.25it/s]

FP: /media/clemens/Maxtor/xeno-canto/fragments/Animalia/Chordata/Aves/Passeriformes/Parulidae/Setophaga/Setophaga ruticilla/
Data: /media/clemens/Maxtor/xeno-canto/
Fragments: /media/clemens/Maxtor/xeno-canto/fragments/
Already downloaded 100113 /media/clemens/Maxtor/xeno-canto/xc/100113.mp3
Already converted 100113 /media/clemens/Maxtor/xeno-canto/xc/100113.wav
/media/clemens/Maxtor/xeno-canto/xc/100113.wav file:///media/clemens/Maxtor/xeno-canto/fragments/Animalia/Chordata/Aves/Passeriformes/Turdidae/Catharus/Catharus guttatus/
Splitting /media/clemens/Maxtor/xeno-canto/xc/100113.wav where energy is below 1.0% for longer than 1.0s.
Finding silences...


100%|██████████| 41/41 [00:00<00:00, 1518.35it/s]

Writing file ./100113_000.wav
Writing file ./100113_001.wav
Writing file ./100113_002.wav
Writing file ./100113_003.wav
Writing file ./100113_004.wav
Writing file ./100113_005.wav
Writing file ./100113_006.wav
Writing file ./100113_007.wav
Writing file ./100113_008.wav
Writing file ./100113_009.wav
Writing file ./100113_010.wav
Writing file ./100113_011.wav
Writing file ./100113_012.wav
Writing file ./100113_013.wav
Writing file ./100113_014.wav
Writing file ./100113_015.wav
Writing file ./100113_016.wav
Writing file ./100113_017.wav
Writing file ./100113_018.wav
Writing file ./100113_019.wav
Writing file ./100113_020.wav
Writing file ./100113_021.wav
Writing file ./100113_022.wav
Writing file ./100113_023.wav
Writing file ./100113_024.wav
Writing file ./100113_025.wav
Writing file ./100113_026.wav
Writing file ./100113_027.wav
Writing file ./100113_028.wav
Writing file ./100113_029.wav
Writing file ./100113_030.wav
Writing file ./100113_031.wav
Writing file ./100113_032.wav
Writing fi

In [ ]:
import XenoCanto as xc
#import importlib; importlib.reload(XenoCanto)

xc.set_dir(project.get_sample_dir())

In [ ]:
for id in df['XC_ID'].head(num_files):
    xc.convert_mp3_to_wav(id)

In [ ]:
from scipy.io import wavfile

In [ ]:
id = '100113'
fs, data = wavfile.read(xc.get_wav_file(id))
print( 'data', data.shape)
print( 'Duration:', data.shape[0] / fs)
print( 'Channels:', data.shape[1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Is this stereo recording useful?

plt.plot(data)
plt.show()

In [ ]:
data[:,0][200000:200100]

In [ ]:
data[:,1][200000:200100]

In [ ]:
# FIX ME: what are min and max

diff = data[:,0] - data[:,1]
diff[200000:200100]

In [ ]:
import numpy as np

plt.plot(diff)
plt.show()

In [ ]:
### https://shallowsky.com/blog/programming/sonograms-in-python.html
# https://matplotlib.org/examples/pylab_examples/specgram_demo.html

Pxx, freqs, bins, im = plt.specgram(data[0:fs*2,0], Fs=5000)#, NFFT=1024, noverlap=900)
plt.show()

In [ ]:
Pxx, freqs, bins, im = plt.specgram(data[:,1], Fs=500)#, NFFT=1024, noverlap=900)
plt.show()

In [ ]:
from scipy import signal
from scipy.io import wavfile
import scipy.io.wavfile

M = 1024

freqs, times, spect = signal.spectrogram(data[:,0], fs=fs, window='hanning',
                                  nperseg=1024, noverlap=M - 100,
                                  detrend=False, scaling='spectrum')

In [ ]:
freqs.size

In [ ]:
freqs[0:20]

In [ ]:
times.size

In [ ]:
plt.plot(spect)
plt.show()